<a href="https://colab.research.google.com/github/alexyin2/LightWill-Project/blob/main/%E5%B0%88%E6%A1%88%E7%A8%8B%E5%BC%8F_%E5%BF%97%E5%B7%A5%E5%A1%AB%E5%AF%AB%E8%A1%A8%E5%96%AE%E6%95%B4%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資料儲存數位化轉型
開發團隊：LightWill \\
開發日期：2022/2/10

## 內容
此程式檔旨在 A) 統計與整合手作步道場次 B) 條列所有手作過得步道 C) 統計步道師實習時數，其步驟包含匯入原始資料、進行必要的資料清理與格式統一化，最後匯出到Google表單，詳細細節請參考下方程式檔。

## 使用說明
1. 
2. 

## 資料格式注意事項
1. worksheet第一列記得填寫欄位名稱
2. 志工人數須為整數，非range
3. 需注意檔案需為google sheet格式，而非.xlsx（在檔名旁邊若有顯示旁邊若有顯示.xlsx就是不對的）

#TODO
加入input box
https://www.w3schools.com/python/showpython.asp?filename=demo_ref_input

## 資料授權 & 輸入必要資訊


In [ ]:
import gspread
import pandas as pd
import numpy as np
import string
import copy
import re

from datetime import datetime, timedelta
from oauth2client.client import GoogleCredentials
from google.colab import auth

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
助教人力需求表URL = input("請輸入「助教人力需求表」的檔案連結：")
手作步道場次URL = input("請輸入「手作步道場次」的檔案連結：")
步道志工URL = input("請輸入「步道志工資料庫」的檔案連結：")
歷年手作步道統計URL = input("請輸入「歷年手作步道統計」的檔案連結：")
YEAR = input("請輸入欲整理資料的西元年（例如：2022）")

# Inputs
# 助教人力需求表URL = 'https://docs.google.com/spreadsheets/d/1fQcbsmv6QT4swdjc7_J4NNlAbVCg0MmybRuktkMlFZk/edit?usp=sharing'
# 手作步道場次URL = 'https://docs.google.com/spreadsheets/d/1hwqmNHHhY7mFW53FyZdHpfMRy5wnP_T0RPdf_TTabXI/edit#gid=0'
# 步道志工URL = 'https://docs.google.com/spreadsheets/d/1BVJiIiRIXodp-rVjRePDKwrQv9slWkTk07dUiS-eLAU'
# 歷年手作步道統計URL = "https://docs.google.com/spreadsheets/d/15O0U8f0P8N-Ur_678lCgYkQ04D5PR_Sg1bmXVKh6Fj8/edit#gid=1778324176"
# YEAR = "2022"

請輸入欲整理資料的西元年（例如：2022）2022


## A. 統計與整合手作步道場次

匯入資料來源：[手作步道活動助教人力需求表 的副本](https://docs.google.com/spreadsheets/d/1lwqKHL3214o3DD-stVLfGMRBfzknQa5CKKKH_pBOZ0c/edit#gid=1043167225)

輸出資料位置：[歷年手作步道統計-2020統計.xlsx 的副本](https://docs.google.com/spreadsheets/d/15O0U8f0P8N-Ur_678lCgYkQ04D5PR_Sg1bmXVKh6Fj8/edit#gid=1778324176)

###1. 檔案讀取

In [ ]:
# Read data
助教人力需求表全 = gc.open_by_url(助教人力需求表URL)
# print("此檔案worksheet:", 助教人力需求表.worksheets())
助教人力需求表 = 助教人力需求表全.worksheet(YEAR).get_all_values()
助教人力需求表 = pd.DataFrame(助教人力需求表[1:], columns=助教人力需求表[0])

print("資料列數:",助教人力需求表.shape[0])
助教人力需求表.head()

資料列數: 8


,日期,活動名稱,手作步道名稱,縣市,鄉鎮區,社區,負責人,志工人數,需求人力,可登錄時數,講師,助教1,助教2,助教3,助教4,助教5,助教6,種子師資實習登記區\n(會依該場活動狀況確認合適的夥伴；實習資格請已通知為準)
0,1/15,五酒桶山工作假期（一）,,,,,朝政,20,除講師外，需2位實習步道師以上(滿),8,伍老師\n文老師,木木,逸婷,,,,,（本場次不開放登記）
1,1/16,五酒桶山工作假期（二）,,,,,朝政,20,除講師外，需2位實習步道師以上(滿),8,伍老師\n文老師,麗娜,隆漢,,,,,（本場次不開放登記）
2,2/9-10,台大梅峰農場手作步道,,,,,可萱,20,4-6(步道師以上1、實習以上2)(滿),16,清地,敬佳,立文,正雄,姿臻,彥輝,慧娟,立文、正雄、姿臻、歸真、彥輝、慧娟、武治
3,3/5-6,玉山林道手作活動(大亞電纜),,,,,可萱,20,6(步道師以上2、實習以上2)(滿),8,伍老師\n銘謙,清地,木木,沂芝,敬佳,武治,,立文、豪萱、秀慈、藝婷、珮雯、武治、靜儀、于盟、美莉、小鈺
4,3/12,B612小行星手作步道體驗（一）,,,,,地瓜,12-15,4-6(請看註解),8,朝政,,,,,,,


### 2. 擷取並整理所需欄位
2.1 其他欄位 \\
2.2 計算助教人數
包括"助教1"至"助教6" & "種子師資實習登記區" 兩種計算的加種
- "助教1"至"助教6"：加總非空白的cell數
- 種子師資實習登記區：若有人--> 人數 = "、"出現數 + 1

In [ ]:
# 新增一個空的新增一個空的df儲存整理完的資料資料
df = pd.DataFrame()

# 活動名稱
df["活動名稱"] = 助教人力需求表["活動名稱"]

# 日期 -> recommend using the format 1/2-1/3 or 1/31-2/2
def clean_date(date_str):
    date_str = date_str.rstrip().replace("\n", "").split('-')[0]
    date = YEAR + "/" + date_str
    date = pd.to_datetime(date, format='%Y/%m/%d').strftime('%Y.%m.%d')
    return date

df['日期'] = 助教人力需求表['日期'].apply(clean_date).astype(str)

# Not working perfectly, will extract 1/15 as 1/1...
# m = re.match(r"([1-9][0-2]?/[1-9][0-2]?)(-?)([1-9]?[0-9]?)", df['日期'])

# 天數 -> asl them to fill it out
df['天數'] = np.nan

# 合作對象 -> Blank, need them to fill it out
df['合作對象'] = np.nan

# 類型 -> Blank, need them to fill it out
df['類型'] = np.nan

# 志工人數 -> need to ba a whole number, not a range
df["志工人數"] = 助教人力需求表["志工人數"]

# 步道名稱 -> ask them to fill it out in the original data file, then we can grab the data
df['步道名稱'] = 助教人力需求表['手作步道名稱']

# 講師
teacher_cols = ['講師'] + [col for col in 助教人力需求表.columns if '助教' in col]
助教人力需求表[teacher_cols] = 助教人力需求表[teacher_cols].replace(r'^\s*$', np.nan, regex=True)

extra_col = '種子師資實習登記區\n(會依該場活動狀況確認合適的夥伴；實習資格請已通知為準)'
助教人力需求表['講師助教'] = (
    助教人力需求表[teacher_cols].notna().sum(axis = 1) + 
    助教人力需求表[extra_col].str.count('、') +
    np.where(助教人力需求表[extra_col].str.contains('、'),1,0)
)

df['講師助教'] = 助教人力需求表['講師助教']

# 備註
df['備註'] = np.nan

###3. 結果呈現

In [ ]:
df

,活動名稱,日期,天數,合作對象,類型,志工人數,步道名稱,講師助教,備註
0,五酒桶山工作假期（一）,2022.01.15,NaN,NaN,NaN,20,,4,NaN
1,五酒桶山工作假期（二）,2022.01.16,NaN,NaN,NaN,20,,4,NaN
2,台大梅峰農場手作步道,2022.02.09,NaN,NaN,NaN,20,,15,NaN
3,玉山林道手作活動(大亞電纜),2022.03.05,NaN,NaN,NaN,20,,17,NaN
4,B612小行星手作步道體驗（一）,2022.03.12,NaN,NaN,NaN,12-15,,2,NaN
5,B612小行星手作步道體驗（二）,2022.03.20,NaN,NaN,NaN,12-15,,2,NaN
6,基隆步道學戶外實作(暖暖),2022.05.21,NaN,NaN,NaN,,,6,NaN
7,旗美社大高雄山區步道學實作(六龜),2022.05.29,NaN,NaN,NaN,,,10,NaN


In [ ]:
# 重新排序欄位順序
col_order = ["活動名稱", "日期", "天數", "合作對象", "類型", "志工人數", "步道名稱", "講師助教", "備註"]
df = df[col_order]

# 將整理好的資料匯入Google表單
新增頁籤名稱名稱 = YEAR+"(From Colab)"
歷年手作步道統計 = gc.open_by_url(歷年手作步道統計URL)

# Creat a new tab if it is not exist already
if not any([i.title == 新增頁籤名稱名稱 for i in 歷年手作步道統計.worksheets()]):
    歷年手作步道統計.add_worksheet(title=新增頁籤名稱名稱, rows=df.shape[0], cols=df.shape[1])

歷年手作步道統計.worksheet(新增頁籤名稱名稱).update([df.columns.values.tolist()] + df.values.tolist())

APIError: ignored

## B. 條列所有參與過的步道

In [ ]:
# Read data
df_trail_google = gc.open_by_url(手作步道場次URL)
values = df_trail_google.worksheet("國內").get_all_values()
df_trail = pd.DataFrame(values[1:], columns=values[0])
df_trail['test'] = 0

# Creat a new tab if it is not exist already
新增頁籤名稱名稱 = "test1"
if not any([i.title == 新增頁籤名稱名稱 for i in df_trail_google.worksheets()]):
    df_trail_google.add_worksheet(title=新增頁籤名稱名稱, rows=df_trail.shape[0], cols=df_trail.shape[1])

df_trail_google.worksheet(新增頁籤名稱名稱).update([df_trail.columns.values.tolist()] + df_trail.values.tolist())

{'spreadsheetId': '1hwqmNHHhY7mFW53FyZdHpfMRy5wnP_T0RPdf_TTabXI',
 'updatedCells': 1216,
 'updatedColumns': 8,
 'updatedRange': 'test1!A1:H152',
 'updatedRows': 152}

## C. 統計步道師實習時數
移除志工重複填寫之時數
1. 檢查名字無空白或其他符號
2. 若同一志工、同一日期且同一參與內容，則僅保留一列(Row)

##### Reference
1. 志工時數登錄表表單: https://docs.google.com/forms/d/1wjXEwwIEZFFRUzgn4FoXTt3T2AQf_m_wES6E7hiRKVw/edit
2. 步道志工時數登錄表: https://docs.google.com/spreadsheets/d/1BVJiIiRIXodp-rVjRePDKwrQv9slWkTk07dUiS-eLAU/edit#gid=1
3. 助教人力需求表: https://docs.google.com/spreadsheets/d/1lwqKHL3214o3DD-stVLfGMRBfzknQa5CKKKH_pBOZ0c/edit#gid=39001012


##### Issue Log
1. 步道志工時數登錄表worksheet "步道志工資料庫"中 欄位"服務時數" 與 "服務時數統計"互相衝突
2. 若一志工重複填寫，但時數上有差異，應取最大值 or 最小值?
3. 發現部分資料填寫日期格式錯誤，例: 2020/08/07-09；建議在表單日期加入"僅填寫1日日期，若連續多日請重新填寫表單"之相關說明


### 1. 資料匯入 & 資料清理

In [ ]:
# Import worksheet
步道志工 = gc.open_by_url(步道志工URL)
步道志工資料庫 = 步道志工.worksheet('步道志工資料庫')
志工時數登錄表 = 步道志工.worksheet('志工時數登錄表')

# Transform to dataframe
步道志工資料庫 = 步道志工資料庫.get_all_values()
步道志工資料庫 = pd.DataFrame(步道志工資料庫[1:], columns=步道志工資料庫[0])

志工時數登錄表 = 志工時數登錄表.get_all_values()
志工時數登錄表 = pd.DataFrame(志工時數登錄表[1:], columns=志工時數登錄表[0])

In [ ]:
志工時數登錄表.head(3)

,時間戳記,志工姓名,服務日期,參與內容,服務地點,參與時數
0,2016/3/10 上午 8:30:04,江逸婷,2016/3/5,協助步道的施做帶領,南澳金岳部落,8
1,2016/6/8 下午 6:56:02,張隆漢,2016/5/28,文山社大海巡署支線步道實作助教,文山區景美山海巡署支線,8
2,2016/6/16 下午 4:47:30,高輔霖,2016/5/21,步道實作帶領,景美山,8


In [ ]:
# 欄位 "志工姓名" 清理

# 建立函式移除空白或其他符號
def name_cleaning(name):
  # remove spaces
  name = name.replace(' ', '')
  # remove special characters
  name = ''.join(i for i in name if i not in string.punctuation)
  # remove numbers
  name = ''.join(i for i in name if not i.isdigit())
  return name

志工時數登錄表['志工姓名'] = 志工時數登錄表['志工姓名'].apply(name_cleaning)


# 測試資料
# test = pd.DataFrame({'志工姓名': ['王小明 ', ' 王@%小明_134', '_王小明.>@']})
# print(test['志工姓名'].apply(name_cleaning))
# del test

In [ ]:
# 欄位 "參與時數" 清理

# 建立函式移除非數值之字符
def hours_cleaning(hours):
  # Only remain numbers
  hours = ''.join(i for i in hours if i.isdigit())
  if hours.isdigit():
    return hours
  
  else:
    # if no numbers in the cell then return 0 hrs
    return '0' 

志工時數登錄表['參與時數'] = 志工時數登錄表['參與時數'].apply(hours_cleaning)

In [ ]:
# 欄位 "服務日期" 清理

# 手動修改row（後續牽扯邏輯較複雜；建議添加表單填寫格式限制）
志工時數登錄表.loc[102, '服務日期'] = '2017/5/6/2017/5/7' # 2017/5/6~20175/7
志工時數登錄表.loc[571, '服務日期'] = '2022/05/09' # 20220/05/09
志工時數登錄表.loc[365, '服務日期'] = '2019/9/7-8' # 林2019/9/7-8
志工時數登錄表.loc[275, '服務日期'] = '2018/9/22/23' # 2018/9/22 23
志工時數登錄表.loc[293, '服務日期'] = '2018/09/21-09/23' # 2018/0921-0923
志工時數登錄表.loc[567, '服務日期'] = '2020/07/11' # 2020/0711
志工時數登錄表.loc[599, '服務日期'] = '2020/05/16' # 2020/05/116
志工時數登錄表.loc[633, '服務日期'] = '2020/09/26-27' # 2020/0926-27
志工時數登錄表.loc[786, '服務日期'] = '2021/8/24-25' # 2021/824-25
志工時數登錄表.loc[938, '服務日期'] = '2022/01/23' # 2022/01/233

志工時數登錄表.loc[780, '服務日期'] = '2021/8/29-31' # 2021/8/29,30,31


# 建立函式將特殊字符統一轉換為'/'
def date_cleaning(date):
  # Remove leading spaces
  date = date.lstrip(' ')
  # transform special characters to '/'
  specialchars = "~-～／.,、_" 
  for specialchar in specialchars:
    date = date.replace(specialchar, '/')
  return date

志工時數登錄表['服務日期'] = 志工時數登錄表['服務日期'].apply(date_cleaning)

### 2. 修改DataFrame以讓每列(Row)資料僅包含一天資訊

In [ ]:
temp = 志工時數登錄表.服務日期.str.split('/', expand=True)
temp, temp1 = temp.loc[:, 1], temp.loc[:, [0, 2, 3, 4, 5]]
# temp.rename(columns={1: 'date2'}, inplace=True)

# 建立函式移除重複之日期
def remove_date_duplicates(date):
  date = list(dict.fromkeys(date))
  return date

temp1 = pd.DataFrame({'date': temp1.apply(remove_date_duplicates, axis=1)})

temp1 = pd.DataFrame(temp1['date'].to_list(), columns=['date1', 'date3', 'date4', 'date5', 'date6'])
temp1.insert(1, "date2", temp)
temp = temp1.fillna(0).astype(int)

# 國曆轉西曆
trans_calen_index = temp[(temp['date1'].astype(str).map(len)) == 3].index.values.astype(int)
temp.loc[trans_calen_index, 'date1'] = temp.loc[trans_calen_index, 'date1'] + 1911
志工時數登錄表.loc[trans_calen_index, '服務日期'] = temp.loc[trans_calen_index, ['date1', 'date2', 'date3']].astype(str).agg('/'.join, axis=1)

In [ ]:
temp.loc[[21, 51, 102, 211], :]

,date1,date2,date3,date4,date5,date6
21,2016,10,8,10,9,0
51,2016,12,2,3,0,0
102,2017,5,6,5,7,0
211,2018,4,27,5,1,0


In [ ]:
# 將欄位 "服務日期" 有跨天數之單一Row，修改為多個Rows(每一Row僅包含一天資訊)

# 針對日期不同
# Create a dataframe to record start and end date
day_diff_index = temp[(temp['date4'] != 0) & (temp['date5'] == 0)].index.values.astype(int)
temp2 = pd.DataFrame()
temp2['date_start'] = temp.loc[day_diff_index, ['date1', 'date2', 'date3']].astype(str).agg('/'.join, axis=1)
temp2['date_end'] = temp.loc[day_diff_index, ['date1', 'date2', 'date4']].astype(str).agg('/'.join, axis=1)
temp2['date_start'] = pd.to_datetime(temp2['date_start'])
temp2['date_end'] = pd.to_datetime(temp2['date_end'])


for i in day_diff_index:
  day = abs(temp2.loc[i, 'date_start'] - temp2.loc[i, 'date_end']).days
  date_record = copy.deepcopy(temp2.loc[i, 'date_start'])
  志工時數登錄表.loc[i, '服務日期'] = temp2.loc[i, 'date_start']
  志工時數登錄表.loc[i, '參與時數'] = (float(志工時數登錄表.loc[i, '參與時數']) / (day + 1))
  # Create rows
  row_copy = copy.deepcopy(pd.Series(志工時數登錄表.loc[i, :]))
  
  for j in range(day):
    date_record += timedelta(days=1)
    row_copy['服務日期'] = date_record
    志工時數登錄表 = 志工時數登錄表.append(row_copy, ignore_index=True)


In [ ]:
# 針對 月份 & 日期不同

# Create a dataframe to record start and end date
month_day_diff_index = temp[temp['date5'] != 0].index.values.astype(int)
temp3 = pd.DataFrame()
temp3['date_start'] = temp.loc[month_day_diff_index, ['date1', 'date2', 'date3']].astype(str).agg('/'.join, axis=1)
temp3['date_end'] = temp.loc[month_day_diff_index, ['date1', 'date4', 'date5']].astype(str).agg('/'.join, axis=1)
temp3['date_start'] = pd.to_datetime(temp3['date_start'])
temp3['date_end'] = pd.to_datetime(temp3['date_end'])

for i in month_day_diff_index:
  day = abs(temp3.loc[i, 'date_start'] - temp3.loc[i, 'date_end']).days
  date_record = copy.deepcopy(temp3.loc[i, 'date_start'])
  志工時數登錄表.loc[i, '服務日期'] = temp3.loc[i, 'date_start']
  志工時數登錄表.loc[i, '參與時數'] = (float(志工時數登錄表.loc[i, '參與時數']) / (day + 1))
  # Create rows
  row_copy = copy.deepcopy(pd.Series(志工時數登錄表.loc[i, :]))
  
  for j in range(day):
    date_record += timedelta(days=1)
    row_copy['服務日期'] = date_record
    志工時數登錄表 = 志工時數登錄表.append(row_copy, ignore_index=True)

### 3. 移除重複資料

In [ ]:
# 統一將欄位 "服務日期" 調整為datetime格式

志工時數登錄表['服務日期'] = pd.to_datetime(志工時數登錄表['服務日期'])

In [ ]:
# 若同一志工、同一日期且同一參與內容，則僅保留一列(Row)

# Show duplicate rows
print(志工時數登錄表.loc[志工時數登錄表.duplicated(subset=['志工姓名', '服務日期', '參與內容'], keep=False), 
                  ['志工姓名', '服務日期', '參與內容', '服務地點', '參與時數']].sort_values(['志工姓名', '服務日期']))

# Remove duplicate rows
志工時數登錄表.drop_duplicates(subset=['志工姓名', '服務日期', '參與內容'], keep='first', inplace=True)

# 將日期轉成字串形式，解決TypeError: Object of type Timestamp is not JSON serializable
志工時數登錄表['服務日期'] = 志工時數登錄表['服務日期'].astype(str)
print(志工時數登錄表.shape)

     志工姓名       服務日期     參與內容             服務地點 參與時數
94    吳清地 2017-03-19   步道實作帶領             澀水社區    8
140   吳清地 2017-03-19   步道實作帶領             澀水社區    8
98    吳清地 2017-03-28   步道實作帶領          嘉明湖國家步道    4
99    吳清地 2017-03-28   步道實作帶領          嘉明湖國家步道    4
100   吳清地 2017-05-13   步道實作帶領         大坑5-1號步道    8
...   ...        ...      ...              ...  ...
709   陳秋坤 2020-12-27   步道實作帶領           新竹石崀古道    8
774   魏薏娟 2021-08-14  步道調查、勘查  東光社區(翠林/水社兩條步道)  4.0
775   魏薏娟 2021-08-14  步道調查、勘查      東光社區(翠林/水社)  4.0
1101  魏薏娟 2021-08-15  步道調查、勘查  東光社區(翠林/水社兩條步道)  4.0
1102  魏薏娟 2021-08-15  步道調查、勘查      東光社區(翠林/水社)  4.0

[87 rows x 5 columns]
(1225, 6)
(1181, 6)


In [ ]:
# 將整理好的資料匯入Google表單
新增頁籤名稱名稱 = "志工時數登錄表(From Colab)"

# Creat a new tab if it is not exist already

if not any([i.title == 新增頁籤名稱名稱 for i in 步道志工.worksheets()]):
    步道志工.add_worksheet(title=新增頁籤名稱名稱, rows=志工時數登錄表.shape[0], cols=志工時數登錄表.shape[1])

步道志工.worksheet(新增頁籤名稱名稱).update([志工時數登錄表.columns.values.tolist()] + 志工時數登錄表.values.tolist())

{'spreadsheetId': '1BVJiIiRIXodp-rVjRePDKwrQv9slWkTk07dUiS-eLAU',
 'updatedCells': 7092,
 'updatedColumns': 6,
 'updatedRange': "'志工時數登錄表(From Colab)'!A1:F1182",
 'updatedRows': 1182}